<a href="https://colab.research.google.com/github/triablomanon/MSE233/blob/HW4/MSE233_HW4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Problem 1. Computing equiblibrium for Prisoner's Dilemma using EXP3 (MWU)

Consider the prisoner's dilema game from [lecture 2 notes](https://www.vsyrgkanis.com/6853sp19/lec2.pdf) for suspect X and suspect Y defined by the payoff matrices:
\begin{align}
X = \begin{pmatrix} -1/2 & -10\\ 0 & -5\end{pmatrix}
\end{align}

\begin{align}
Y = \begin{pmatrix} -1/2 & 0\\ -10 & -5\end{pmatrix}
\end{align}

As described in lecture notes the pure Nash Equilibrium is when both prisoners choose the second action available to them (betray). Note importantly if these are the payoff matrices, then the loss matrices will be the complements (i.e. -X and -Y).

Implement exponential weight updates algorithm and see if using no-regret dynamics where both players use the EXP algorithm to update their choice probabilities at each step, converges to an approximate Nash equilibrium of the prisoner's dilemma game. Use the template code we provide below and fill in the parts that we left as `{{FILL IN}}`

In [1]:
import numpy as np
import matplotlib.pyplot as plt
np.random.seed(42)

Here we define a generic two player game interface so we can resuse it throughout the homework.

In [2]:
class TwoPlayerGameInterface:
    """
    Generic Interface for a two player game
    _numActions : no. of actions for each of the players
    _numPlayers : no. of players in the game
    _X : loss matrix for x player
    _Y : loss matrix for y player
    """
    def __init__(self, X, Y):
        self._numActions = X.shape
        self._numPlayers = len(self._numActions)
        self._X = X
        self._Y = Y
        print(f'Created a game with {self._numPlayers} players',
              f'and number of actions: {self._numActions}')

    def getPlayerLoss(self, i : int):
        if i == 0:
            return self._X
        if i == 1:
            return self._Y

    def getNumActions(self):
        return self._numActions

    def getNumPlayers(self):
        return self._numPlayers

## Problem 1.1
This is the exponential weight updates algorithm as in HW1. Everything is very similar to HW1, we just want you to modify it slightly to generalize it for any two player game with two different loss matrices, instead of a zero-sum game as in HW1. Moreover, instead of keeping track at every period of the distribution of actions of the players, we will be sampling a strategy based on the distribution and giving feedback to the players based on the sampled strategies, and not the expected feedback, over the distribution of the opponent's strategies.

In [4]:
def playNoRegretGame(twoPlayerGame):
    """
    (1.5 points)
    Generic No Regret EXP algorithm for a two player game, similar to HW1
    but not necessarily for a zero sum game.
    """

    # number of actions of each player
    nActions = twoPlayerGame.getNumActions()

    # numberof iterations of no-regret dynamics
    T = 10000
    # step size for exponential weights
    etas = 10 * np.sqrt(np.log(nActions)/(2 * T))

    # initialize to random initial probabilities over actions to avoid symmetry
    x = np.random.uniform(.5, 1.5, size=nActions[0]) * np.ones((T,nActions[0]))
    x /= np.sum(x, axis=-1, keepdims=True)
    y = np.random.uniform(.5, 1.5, size=nActions[1]) * np.ones((T,nActions[1]))
    y /= np.sum(y, axis=-1, keepdims=True)

    # Get player LOSS matrices
    X = twoPlayerGame.getPlayerLoss(0)
    Y = twoPlayerGame.getPlayerLoss(1)

    strategyX, strategyY = np.zeros(T-1).astype(int), np.zeros(T-1).astype(int)
    for t in np.arange(1, T):
        # sample an action for the x and y player
        strategyX[t-1] = np.random.choice(nActions[0], p=x[t-1])
        strategyY[t-1] = np.random.choice(nActions[1], p=y[t-1])

        # loss vector for x player, given last choice of y
        lx = X[:,strategyY[t-1]]
        # loss vector for y player, given last choice of x
        ly = Y[strategyX[t-1],:]

        # update probabilities for x player, based on Exponential Weight Updates
        x[t] = x[t-1] * np.exp(-etas[0] * lx) / np.sum(x[t-1] * np.exp(-etas[0] * lx))
        # update probabilities for y player, based on Exponential Weight Updates
        y[t] = y[t-1] * np.exp(-etas[1] * ly) / np.sum(y[t-1] * np.exp(-etas[1] * ly))

    return strategyX, strategyY

In [5]:
def run_prisoners_dilemma_game(X, Y, TwoPlayerGameInterface=TwoPlayerGameInterface, playNoRegretGame=playNoRegretGame):
  '''
  (0.5 point)
  Run the prisoner's dilemma game by using TwoPlayerGameInterface and playNoRegretGame.
  Inputs: X,Y: loss matrices
  Returns: xbar, ybar: equilibrim startegy for x and y players
  '''
  prisonersDilemmaGame = TwoPlayerGameInterface(X, Y)
  strategyX, strategyY = playNoRegretGame(prisonersDilemmaGame)

  # calculate equilibrium as the pair of the empirical distribution of
  # choices of x player and the empirical distribution of choices of the y player
  # See rho^T in slide 61 of Lecture 7
  xbar = np.zeros(X.shape[0])
  ybar = np.zeros(Y.shape[1])
  for i in strategyX:
    xbar[i] += 1
  for i in strategyY:
    ybar[i] += 1
  xbar /= len(strategyX)
  ybar /= len(strategyY)
  #xbar = np.bincount(strategyX, minlength=X.shape[0]) / len(strategyX)
  #ybar = np.bincount(strategyY, minlength=Y.shape[1]) / len(strategyY)
  return xbar, ybar

In [6]:
# LOSS matrix for player x
X = np.array([[1/2, 10], [0, 5]])

# LOSS matrix for player y
Y = np.array([[1/2, 0], [10, 5]])

xbar, ybar = run_prisoners_dilemma_game(X, Y)

with np.printoptions(precision=3, suppress=True):
    print(f"P1 distribution of strategies: {xbar}")
    print(f"P2 distribution of strategies: {ybar}")

Created a game with 2 players and number of actions: (2, 2)
P1 distribution of strategies: [0. 1.]
P2 distribution of strategies: [0.001 0.999]


Expected answer: \
P1 distribution of strategies: [0. 1.] \
P2 distribution of strategies: [0.001 0.999]

(0.5 points) Explain whether the marginal empirical distributions of each player's choice probabilities converged to a Nash equilibrium of the prisoner's dilemma game?

Given the loss matrices for the two players, we can see that the marginal empirical distributions of the player's choice probabilities **DID converge** to a Nash equilibrium:

Indeed for Player 2, strategy 2 is dominant:
- loss 0 vs. 1/2 when P1 plays strategy 1
- loss 5 vs 10 when P1 plays strategy 2

Therefore, [0, 1] for both P1 and P2 is a Nash equilibrium, and the algorithm has reached a distribution of [0, 1] for P1 and [0.001, 0.999] for P2, which is satisfyingly close.



# Problem 2. No-Swap Regret Dynamics

## Problem 2.1 Implementing No-Swap Regret Dynamics

In Lectures 7 & 8, we explored how no-swap regret dynamics are stronger than no-regret dynamics. The joint empirical distribution of no-regret dynamics rapidly converges to coarse-correlated equilibria in arbitrary games (see e.g. the [Lecture Notes](https://www.vsyrgkanis.com/6853sp19/lecture06.pdf) and the corresponding class [Lecture 7, Presentation](https://raw.githubusercontent.com/stanford-msande233/spring25/master/assets/presentations/Lecture7.pdf).  In no-swap regret dynamics, the learning algorithm instead converges to a correlated equilibrium. For details beyond what was covered in [Lecture 7](https://raw.githubusercontent.com/stanford-msande233/spring25/master/assets/presentations/Lecture7.pdf). More specifically, no-swap regret describes how we wouldn't prefer a strategy by which we change an action to some other action given some deterministic fixed mapping. Clearly no-regret is weaker than this because no-regret is saying we wouldn't prefer mapping every action we took to a single action, which is encompassed by no-swap. In this part of the assignment we will be implementing no-swap regret dynamics via the no-regret to no-swap regret reduction described in the [lecture notes](https://www.vsyrgkanis.com/6853sp19/lecture06.pdf) and in [Lecture 8](https://raw.githubusercontent.com/stanford-msande233/spring25/master/assets/presentations/Lecture8.pdf).

### Setting up the no-swap no-regret reduction

The following code sets up the no-swap no-regret reduction setting up each player's choice probabilities (playerStrategies) and the choice probabilies for each no-regret algorithm that each player has access to (algoStrategies). Note that the number of algorithms each player has is equal to the number of actions available to them. You should read this code to understand how each of these data structures is initailized but DO NOT MODIFY THIS CODE.

In [ ]:
# Initializing RPS (Rock-Paper-Scissors) game
def initializeRPSGame(rpsGame : TwoPlayerGameInterface):
    '''
    Create initiatlization for an Rock-Paper-Scissors (RPS) game.
    returns:
    T: total # of time periods
    etas: learning constant
    algoStrategies: algorithm strategy initialization
    nActions: # of actions
    nplayers: # of players
    '''
    # Note that algoStrategies[0] entries are the algorithm weights for player 1
    # for each time t. algoStrategies[1] entries are the corresponding
    # for player 2.
    # algoStrategies[0][0] for example are player 1's algorithm weights
    # at time t = 0.
    T = 10000 # number of learning periods
    algoStrategies, playerStrategies = {}, {}
    nActions = rpsGame.getNumActions()
    nplayers = rpsGame.getNumPlayers()

    etas = 10 * np.sqrt(np.log(nActions) / (2 * T))

    for i in range(nplayers):
        # weights of all algorithm instances of player i at each learning period
        # we add some randomness in the initialization to break the symmetry
        # of the dynamics
        rand_init = np.random.uniform(.5, 1.5, size=(nActions[i], nActions[i]))
        algoStrategies[i] = rand_init * np.ones((T, nActions[i], nActions[i]))
        algoStrategies[i] /= np.sum(algoStrategies[i], axis=-1, keepdims=True)

    return T, etas, algoStrategies, nActions, nplayers

In [ ]:
def solveForQ(weights):
    """
    Determines the player's probability distribution by finding the eigenvectors
    of the player's algorithm weights. Note an eigenvector of a matrix is a vector
    that when multiplied by a matrix, yields itself multiplied by some scalar
    (the eigenvalue). Here we return the eigenvector whose eigenvalue is 1.
    """
    evals, evecs = np.linalg.eig(weights.T)
    evec1 = evecs[:, np.isclose(evals, 1)]
    evec1 = evec1[:, 0]
    # eigs finds complex eigenvalues and eigenvectors, so you want the real part
    real_evec1 = np.abs(evec1.real)
    stationary = real_evec1 / np.sum(real_evec1)
    return stationary

### No-swap regret dynamics algorithm
Implement the black box reduction of no-regret to no-swap regret. Use multiplicative weight updates and compute an approximate equilibrium of the game using no-swap regret dynamics where both players use the MWU algorithm to update their choice probabilities of their no-regret algorithms at each step.

Consult Slides 61-64 from [Lecture 8](https://raw.githubusercontent.com/stanford-msande233/spring25/master/assets/presentations/Lecture8.pdf)

In [ ]:
def playNoSwapRegretGame(rpsGame : TwoPlayerGameInterface):
    '''
    (3 points)
    Implement the no-swap-regret algorithm for the rps game defined by a TwoPlayerGameInterface.
    Returns: strategyX, strategyY: equilibrium strategies for players x and y
    '''
    T, etas, algoStrategies, nActions, nPlayers = initializeRPSGame(rpsGame)
    strategyX, strategyY = np.zeros(T-1).astype(int), np.zeros(T-1).astype(int)
    # Iterate over learning periods
    for t in np.arange(1, T):
        # Determine the probability distribution q over actions for each player
        qX = solveForQ(algoStrategies[0][t-1])
        qY = solveForQ(algoStrategies[1][t-1])

        # Pick action to play for player 1 based on player weights
        strategyX[t-1] = np.random.choice(nActions[0], p=qX)
        # Pick action to play for player 2 based on player weights
        strategyY[t-1] = np.random.choice(nActions[1], p=qY)

        # Get the loss matrix for player 1
        lossMatrixX = rpsGame.getPlayerLoss(0)
        # Get the payoff from opponent's action
        actionLossX = lossMatrixX[:, strategyY[t-1]]
        # Update the algorithm weights based on loss and player strategies
        for j in np.arange(nActions[0]):
            # calculate perceived loss vector for the Aj instance
            # of the EXP algorithm
            perceivedLossXj = actionLossX * qX[j]
            # update the Aj algorithm weights using the EXP rule
            algoStrategies[0][t, j, :] = algoStrategies[0][t-1, j, :] * np.exp(-etas[0] * perceivedLossXj) / np.sum(algoStrategies[0][t-1, j, :] * np.exp(-etas[0] * perceivedLossXj))


        # Get the loss matrix for player 1
        lossMatrixY = rpsGame.getPlayerLoss(1)
        # Get the payoff from opponent's action
        actionLossY = lossMatrixY[strategyX[t-1], :]

        # Update the algorithm weights based on loss and player strategies
        for j in np.arange(nActions[1]):
            # calculate perceived loss vector for the Aj instance
            # of the EXP algorithm
            perceivedLossYj = actionLossY * qY[j]
            # update the Aj algorithm weights using the EXP rule
            algoStrategies[1][t, j, :] = algoStrategies[1][t-1, j, :] * np.exp(-etas[1] * perceivedLossYj) / np.sum(algoStrategies[1][t-1, j, :] * np.exp(-etas[1] * perceivedLossYj))

    return strategyX, strategyY

### Problem 2.2 Calculate the empirical joint distribution (equilibrium).


Using the choice probabilies of each player, determine empirical joint distribution (i.e. the quantity $\pi^T$ in [Lecutre 7](https://raw.githubusercontent.com/stanford-msande233/spring25/master/assets/presentations/Lecture7.pdf), Slides 61). This distribution should be an approximate Correlated Equilibrium (see [Lecture 8, Slide 66](https://raw.githubusercontent.com/stanford-msande233/spring25/master/assets/presentations/Lecture8.pdf))

In [ ]:
def calculateEquilibrium(rpsGame : TwoPlayerGameInterface, strategyX, strategyY):
    """
    (1 point)
    QJoint represents the mean likelihood of action x,y from player X,Y.
    Moreover for a 2 player game with finite actions (N,M) for player X and
    player Y respecitively. QJoint will be an NxM matrix i.e. same size as
    the payoff matrix.
    """
    # Determining the shape of the joint distribution from the loss matrix
    qjoint = np.zeros(rpsGame.getPlayerLoss(0).shape)
    for index in np.ndindex(qjoint.shape):
        # Determine the empirical probability of action pairs of the two players
        # Note index is a the tuple (x,y) from (0,0) -> (n,m)
        n, m = index
        qjoint[index] = np.mean((strategyX == n) & (strategyY == m))
    return qjoint

### Problem 2.3
Calculate the swap regret at the calculated Approximate Correlated Equilibrium  for each pair of strategies $j,j'$ of each player. Consult with [Lecture 8, Slide 66](https://raw.githubusercontent.com/stanford-msande233/spring24/master/assets/presentations/Lecture8.pdf). Then calculate the maximum such regret over deviating strategies $j'$ for each action $j$.

In [ ]:
def calculateSwapRegret(rpsGame : TwoPlayerGameInterface, qjoint):
    '''
    (2 points)
    Compute and print the swap regret given an instance of a game defined by TwoPlayerGameInterface
    and qjoint returned by calculateEquilibrium.
    '''

    # get the loss matrices for each player and the number of actions of each
    # player
    lossX = rpsGame.getPlayerLoss(0)
    lossY = rpsGame.getPlayerLoss(1)
    n, m = rpsGame.getNumActions()

    # Calculate and print swap regrets for player X
    for j in range(n):
        # loss for player x when playing algorithm j
        loss_j_x = np.sum(lossX[j, :] * qjoint[j, :])
        max_regret_j_x = -np.inf
        for jp in range(n):
            loss_jp_x = np.sum(lossX[jp, :] * qjoint[j, :])
            regret_j_jp_x = loss_j_x - loss_jp_x
            max_regret_j_x = max(max_regret_j_x, regret_j_jp_x)
        print(f"Player X, Action {j}: {max_regret_j_x:.4f}")


    # Calculate and print swap regrets for player Y
    for j in range(m):
        # loss for player y when playing algorithm j
        loss_j_y = np.sum(lossY[:, j] * qjoint[:, j])
        max_regret_j_y = -np.inf
        for jp in range(m):
            loss_jp_y = np.sum(lossY[:, jp] * qjoint[:, j])
            regret_j_jp_y = loss_j_y - loss_jp_y
            max_regret_j_y = max(max_regret_j_y, regret_j_jp_y)
        print(f"Player Y, Action {j}: {max_regret_j_y:.4f}")

## Problem 2.4. Regular Rock-Paper-Scissors (RPS) Game

For regular RPS we have a simple payoff bi-matrix (which we define as two separate matrices X & Y) for player x and player y. Here rock, paper, scissors correspond to indices 0, 1, 2 respectively. Observe that this is a zero-sum game with payoff matrices

\begin{align}
X = \begin{pmatrix} 0 & -1 & 1\\ 1 & 0 & -1\\ -1 & 1 & 0\end{pmatrix}
\end{align}

\begin{align}
Y = \begin{pmatrix} 0 & 1 & -1\\ -1 & 0 & 1\\ 1 & -1 & 0\end{pmatrix}
\end{align}

In this section we compare the no-swap regret dynamics of the regular RPS game to the no-regret dyamics. Note that this game DOES converge to a correlated equilibrium!

### First we define the regularRPS game using our TwoPlayerGameInterface



In [ ]:
# Define the regular RPS game using loss matrices (negative of payoff matrices)
regularRPSXLoss = - np.array([[0, -1,  1],
                              [1,  0, -1],
                              [-1, 1,  0]])
regularRPSYLoss = - regularRPSXLoss
regularRPSGame = TwoPlayerGameInterface(regularRPSXLoss, regularRPSYLoss)

Created a game with 2 players and number of actions: (3, 3)


### Analysis of the no-swap regret dynamics v.s. no-regret dynamics of regular RPS game.
In this section we compare the no-swap regret dynamics on the Regular RPS game to the no-regret dynamics of the Regular RPS game.



#### No-swap regret Dynamics of Regular RPS

In [ ]:
np.random.seed(123567)

# Play Regular RPS game with No Swap Regret Dynamics
regularRPSactionsX, regularRPSactionsY = playNoSwapRegretGame(regularRPSGame)

# calculate empirical joint distribution
qjoint = calculateEquilibrium(regularRPSGame, regularRPSactionsX, regularRPSactionsY)
with np.printoptions(precision=3):
    print(f"Empirical joint Distribution:\n {qjoint}\n")

# The empirical joint distribution should be an approximate correlated
# equilibrium. We verify that by calculating the swap regret
# using the function you wrote earlier
print('Swap regrets are:')
calculateSwapRegret(regularRPSGame, qjoint)

Empirical joint Distribution:
 [[0.116 0.112 0.111]
 [0.108 0.114 0.111]
 [0.109 0.11  0.109]]

Swap regrets are:
Player X, Action 0: 0.0050
Player X, Action 1: 0.0097
Player X, Action 2: 0.0000
Player Y, Action 0: 0.0052
Player Y, Action 1: 0.0003
Player Y, Action 2: 0.0022


Expected answers: \
Empirical joint Distribution: \
 [[0.116 0.112 0.111] \
 [0.108 0.114 0.111] \
 [0.109 0.11  0.109]] \

Swap regrets are \
Player X, Action 0: 0.0050 \
Player X, Action 1: 0.0097 \
Player X, Action 2: 0.0000 \
Player Y, Action 0: 0.0052 \
Player Y, Action 1: 0.0003 \
Player Y, Action 2: 0.0022 \


#### No-regret Dynamics of Regular RPS

In [ ]:
np.random.seed(123567)

# Play Regular RPS game with No Regret Dynamics
regularRPSactionsX, regularRPSactionsY = playNoRegretGame(regularRPSGame)

# calculate empirical joint distribution
qjoint = calculateEquilibrium(regularRPSGame, regularRPSactionsX, regularRPSactionsY)
with np.printoptions(precision=3):
    print(f"Empirical joint Distribution:\n {qjoint}\n")

# Calculate the swap regret using the function you wrote earlier
print('Swap regrets are:')
calculateSwapRegret(regularRPSGame, qjoint)

Empirical joint Distribution:
 [[0.011 0.165 0.162]
 [0.158 0.011 0.161]
 [0.155 0.167 0.011]]

Swap regrets are:
Player X, Action 0: 0.1575
Player X, Action 1: 0.1528
Player X, Action 2: 0.1310
Player Y, Action 0: 0.1499
Player Y, Action 1: 0.1584
Player Y, Action 2: 0.1519


Expected Answers: \

Empirical joint Distribution: \
 [[0.011 0.165 0.162] \
 [0.158 0.011 0.161] \
 [0.155 0.167 0.011]] \

Swap regrets are:  \
Player X, Action 0: 0.1575 \
Player X, Action 1: 0.1528 \
Player X, Action 2: 0.1310 \
Player Y, Action 0: 0.1499 \
Player Y, Action 1: 0.1584 \
Player Y, Action 2: 0.1519 \

### Question:
(1 point) Comment on the differences between the swap-regret when running no-swap regret dynamics and no-regret dynamics for regular RPS. Does the empirical joint distribution converge to a correlated equilibrium under the swap-regret dynamics? Does the empirical joint distribution converge to a correlated equilibrium under the no-regret dynamics? Do they converge to some other relaxed notion of a correlated equilibrium?

###Answer

We can see that as we expect, running the no-swap regret dynamics leads to swap regrets that are much closer to 0 than running the standard no-regret dynamics for the Rock Paper Scissors game.

We also see that the no-regret dynamics do not lead to a correlated equilibrium for the empirical joint distribution. For example for Player 1 starting with action 0 (Rock):

Empirical joint Distribution:
 [[0.011 0.165 0.162]
 [0.158 0.011 0.161]
 [0.155 0.167 0.011]]

Switching the actions 0 (Rock) to action 2 (Scissors) would lead to a gain in utility of -1\*0.011 + 1\*0.165 + 0*0.162 ~ 0.154 > 0

So by checking this one swap we can see that we are very far from a correlated equilibrium.

From lecture 7 we know that we instead reach a **coarse correlated equilibrium** where the expected utility over all swaps from a given action converges to 0, but not individual swaps.